# Data Processing
In this notebook, I will be loading the data necessary for this project, combining the data sources and performing initial data processing steps. I will also be performing some exploratory data analysis for the purposes of identifying missing values and outliers, which will then be followed up with the appropriate processing steps.

In [3]:
%load_ext autoreload
%autoreload 2
import os
from tqdm import tqdm
import pickle
tqdm.pandas()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from data.util.paths import DATA_PATH
from lib.processing import load_from_pkl
from lib.pipelines import extracted_pipe, api_pipe

## Introduction

The data for this project has been sourced from Discogs.com, the largest online marketplace for second-hand physical music. There are three components to the data that will make up our final dataset, namely:
1. api_data -> Data taken from the official Discogs.com API (https://www.discogs.com/developers)
2. extracted_data -> Data scraped from Discogs release pages
3. image_embedding_data -> High-Level Features extracted from the cover images of each Album

At this juncture, we will load, investigate and transform the first two data sources outlined above. The data was collected continuously over the months of February and March 2020 using the scripts found in the data directory of this project. To scrape each of these three data sources, the main() function of the data/main.py file was used. It is not recommended to run this script oneself, as it will require investing into a paid proxy service due to the rate limiting in force by Discogs.com. The data has been made available for the EPFL Extension School Reviewers at INSERT_DATA_SOURCE_HERE. For more details on how each data source was filtered and scraped, please refer to the classes and functions contained in the data/util/scrape.py module.
    
First, we will load the data extracted from the Discogs.com API, henceforth referred to as ``api_data``

## API Data
### Loading and Overview

In [6]:
api_data = load_from_pkl('api',path=DATA_PATH)

In [7]:
api_data.head()

,id,release_id,title,year,country,genre,style,label,community_have,community_want,formats,master_id,thumb_url,release_url
0,1,11918321,Bing Crosby - Crosbyana,1934,Canada,b'\x80\x03]q\x00(X\x04\x00\x00\x00Jazzq\x01X\x...,b'\x80\x03]q\x00(X\x08\x00\x00\x00Big Bandq\x0...,b'\x80\x03]q\x00(X\x05\x00\x00\x00Deccaq\x01X\...,1,2,b'\x80\x03]q\x00}q\x01(X\x03\x00\x00\x00qtyq\x...,1354381,https://img.discogs.com/J7vwmOhWMdUJ5vYuaYZvIj...,https://api.discogs.com/releases/11918321
1,2,10550056,Bing Crosby - Crosbyana,1934,US,b'\x80\x03]q\x00(X\x04\x00\x00\x00Jazzq\x01X\x...,b'\x80\x03]q\x00(X\x08\x00\x00\x00Big Bandq\x0...,b'\x80\x03]q\x00X\x05\x00\x00\x00Deccaq\x01a.',4,4,b'\x80\x03]q\x00}q\x01(X\x03\x00\x00\x00qtyq\x...,1354381,https://img.discogs.com/aLpqYUso3yY53XDHwiqqB-...,https://api.discogs.com/releases/10550056
2,3,6910984,Tommy Dorsey And His Clambake Seven - Tommy Do...,1935,US,b'\x80\x03]q\x00X\x04\x00\x00\x00Jazzq\x01a.',b'\x80\x03]q\x00X\x05\x00\x00\x00Swingq\x01a.',b'\x80\x03]q\x00(X\r\x00\x00\x00Swing Classicq...,4,2,b'\x80\x03]q\x00}q\x01(X\x0c\x00\x00\x00descri...,0,https://img.discogs.com/FkuaZ7cqdBt5-TvS2ck0hP...,https://api.discogs.com/releases/6910984
3,4,12959431,"Paul Whiteman And His Orchestra, Bix Beiderbec...",1936,US,b'\x80\x03]q\x00X\x04\x00\x00\x00Jazzq\x01a.',b'\x80\x03]q\x00X\x05\x00\x00\x00Swingq\x01a.',b'\x80\x03]q\x00X\x06\x00\x00\x00Victorq\x01a.',3,3,b'\x80\x03]q\x00}q\x01(X\x0c\x00\x00\x00descri...,0,https://img.discogs.com/Wg31rg_x6TtOMz-jIDC-3l...,https://api.discogs.com/releases/12959431
4,5,4453491,Jimmie Lunceford And His Orchestra - For Dance...,1937,Canada,b'\x80\x03]q\x00X\x04\x00\x00\x00Jazzq\x01a.',b'\x80\x03]q\x00.',b'\x80\x03]q\x00(X\x05\x00\x00\x00Deccaq\x01X\...,5,2,b'\x80\x03]q\x00}q\x01(X\x0c\x00\x00\x00descri...,1519538,,https://api.discogs.com/releases/4453491


Above we see the data saved from the querying of the Discogs.com API. The data includes all available jazz albums from the API and retains the columns that were deemed to be the most relevant and informative for the purposes of estimating the market value of each record. As we can see from the above calling of the ``.head()`` method on the api_data DataFrame, several columns have been saved in a binary format. This is due to the fact that the API often returned multiple categories for these columns, and as such they were initially saved in a list in the process of scraping. As this data comes from a SQL database, it has been pickled to enable the storage of the data. Let us now unpickle this data in order to get a better understanding of the structure of the ``api_data``.

In [8]:
api_data.applymap(lambda x: pickle.loads(x) if isinstance(x,bytes) else x).head()

,id,release_id,title,year,country,genre,style,label,community_have,community_want,formats,master_id,thumb_url,release_url
0,1,11918321,Bing Crosby - Crosbyana,1934,Canada,"[Jazz, Pop]","[Big Band, Vocal]","[Decca, The Compo Company Ltd.]",1,2,"[{'qty': '6', 'descriptions': ['10""', '78 RPM'...",1354381,https://img.discogs.com/J7vwmOhWMdUJ5vYuaYZvIj...,https://api.discogs.com/releases/11918321
1,2,10550056,Bing Crosby - Crosbyana,1934,US,"[Jazz, Pop]","[Big Band, Vocal]",[Decca],4,4,"[{'qty': '6', 'descriptions': ['10""', '78 RPM'...",1354381,https://img.discogs.com/aLpqYUso3yY53XDHwiqqB-...,https://api.discogs.com/releases/10550056
2,3,6910984,Tommy Dorsey And His Clambake Seven - Tommy Do...,1935,US,[Jazz],[Swing],"[Swing Classic, RCA Victor]",4,2,"[{'descriptions': ['10""', '78 RPM', 'Album', '...",0,https://img.discogs.com/FkuaZ7cqdBt5-TvS2ck0hP...,https://api.discogs.com/releases/6910984
3,4,12959431,"Paul Whiteman And His Orchestra, Bix Beiderbec...",1936,US,[Jazz],[Swing],[Victor],3,3,"[{'descriptions': ['10""', '78 RPM', 'Album', '...",0,https://img.discogs.com/Wg31rg_x6TtOMz-jIDC-3l...,https://api.discogs.com/releases/12959431
4,5,4453491,Jimmie Lunceford And His Orchestra - For Dance...,1937,Canada,[Jazz],[],"[Decca, Decca Records, Inc.]",5,2,"[{'descriptions': ['10""', '78 RPM', 'Album'], ...",1519538,,https://api.discogs.com/releases/4453491


As all columns are now human-readable, we can discuss each column and its relevance for the purposes of our goal of estimating the market value of jazz albums on the basis of their meta-characteristics
- ``release_id`` and ``master_id``
    - These columns represent the unique identifier of a specific release of a record on Discogs.com, and the unique identifier of the album irrespective of the release, respectively.
    - While ``release_id`` will be of no use to us in the estimation of album market value, it is valuable as a primary key for the joining of the data extracted from the Discogs.com API with the scraped release page and image data we will be introducing later
    - ``master_id`` will be useful in constructing linkages between entries that are the same album, but released in a different country, format, or by another label
- ``country``
    - This feature gives us an insight into which country the record was released from
    - While not immediately obvious from the snapshot above, this feature can have multiple countries for its value, such as "England & USA", which requires a more thoughtful approach than just direct one-hot encoding in order to optimally preserve the information it encodes
- ``title``
    - This feature includes both the title of the album, as well as the title of the artist
    - It will be necessary to split this feature into two independent ones, such that entries can be linked to others by virtue of having the same authoring artist or group
- ``community_want`` and ``community_have``
    - These features outline the registered demand and supply, respectively for given albums on the Discogs.com platform
    - Users can register their ownership or desire of a certain album, which is then aggregated over users and recorded in these features
    - These are the first features we see which will not be taken into account in the record store estimation scenario, as in a physical visit to the record store, it is not possible to know exactly how many people have and want a specific record
- ``genre``, ``style`` and ``label``
    - These features outline the genres, styles and labels associated with each album release
    - As has been alluded to previously, there are potentially multiple values for this feature, which will make standard encoding methods such as One-Hot Encoding difficult to rationalize
- ``formats``
    - Taken directly from the Discogs.com API, the formats feature is the most unruly of all, as it contains up to 4 sub-features,namely:
        - Format Description
            - Album release specifics
            - Examples
                - '10"', '78RPM', 'Album', 'Reissue'
        - Format Text
            - Additional free form notes associated with the record release
            - Examples
                - 'Paper Sleeve', 'Red Vinyl'
        - Format Name
            - The name of the format
            - Examples
                - 'CD', 'Vinyl', 'Cassette'
        - Format Quantity
            - An integer representing how many units of the format are associated with a purchase of the release
- ``thumb_url`` and ``release_url``
    - These features are unnecessary for the purposes of our analysis here, but were useful in order to conduct the data extraction of the ``extracted_data`` and ``image_embedding_data`` obtained from the Discogs.com platform itself

### Investigation and Transformation

Having taken an initial look at ``api_data``, we can now move on to the cleaning and transformation of this DataFrame through a previously written pipeline, to be found in the lib/pipelines.py module. For the purposes of exposition, the pipeline will be replicated here step-by-step, with motivation behind each step. In the following notebooks, a direct import of the pipeline will be used in lieu of a replication as below.

Below the required transformers will be imported and composed into the pre-defined pipeline structure, after which the pipeline will be explained in its entirety.

In [9]:
from sklearn.pipeline import Pipeline
from lib.transformers import (ColumnRemover, 
                              TitleSplitter, 
                              Unpickler, 
                              LabelCleanReduce, 
                              ArtistCleanReduce, 
                              DuplicateRemover, 
                              CountryEncoder, 
                              GenreEncoder, 
                              MultiValueCategoricalEncoder, 
                              FormatEncoder, 
                              FormatTextCleanReduce, 
                              TimePeriodEncoder,
                              OutlierRemover)

In [10]:
clean_text_pipe = Pipeline([
    #Step 4a - Homogenize and clean the label feature
    ('label',LabelCleanReduce()),
    #Step 4b - Homogenize and clean the artist feature
    ('artist',ArtistCleanReduce())
])

column_encoding_pipe = Pipeline([
    #Step 6a - Identify regions and superregions associated with countries and approrpiately encode
    #multi-country album releases
    ('country',CountryEncoder()),
    #Step 6b - Expand and encode the genre feature
    ('genre',GenreEncoder()),
    #Step 6c - Expand and encode the style feature
    ('style',MultiValueCategoricalEncoder(feature='style'))
])

format_pipe = Pipeline([
    #Step 7a - Create the format_description, format_text, format_name and format_quantity features
    ('make_columns',FormatEncoder()),
    #Step 7b - Remove format_quantity outliers
    ('remove_quantity_outliers', OutlierRemover('format_quantity')),
    #Step 7c - Expand and encode the format_description feature
    ('encode_descriptions',MultiValueCategoricalEncoder('format_description')),
    #Step 7d - Homogenize and clean the format_text feature
    ('clean_format_text',FormatTextCleanReduce())
])

api_pipe = Pipeline([
    #Step 1 - Remove Unnecessary Columns
    ('remove_columns',ColumnRemover(['id','thumb_url','release_url'])),
    #Step 2 - Splitting title Feature into title and artist
    ('split_title',TitleSplitter()),
    #Step 3 - Unpickle Pickled Features
    ('unpickle',Unpickler(['genre','style','label','formats'])),
    #Step 4 - Clean Features with Multiple Categorical Values per Sample and High Categorical Variance
    ('clean_text',clean_text_pipe),
    #Step 5 - Remove Entries with Duplicate release_ids
    ('remove_duplicates',DuplicateRemover('release_id')),
    #Step 6 - Encode Categorical Features which cannot be encoded conventionally
    ('encode_columns',column_encoding_pipe),
    #Step 7 - Expand the 'format' Feature from Dictionary to Column Format
    ('format_columns',format_pipe),
    #Step 8 - Encode Jazz Periods and Eras associated with the Release Year of each Album
    ('encode_time_periods',TimePeriodEncoder())
])

### Pipeline Step Exposition
#### Step 1 - Removing Unnecessary Columns
This step is mostly self-explanatory. For the purposes of this analysis, neither ``thumb_url`` and ``release_url`` will be relevant, and the ``id`` column is superfluous
#### Step 2 - Splitting ``title`` Feature into ``title`` and ``artist``
As was alluded to in the initial ``api_data`` overview, the original ``title`` feature clearly contains two distinct pieces of information, namely the name of the performing artist or group for a given release, as well as the name of said release. As such, with this transformer, we split the title feature according to the common delimiter in this column, namely a hyphen ('-') in order to yield an individual ``title`` and ``artist`` column. An equivalent operation is shown below on a sample of the dataset.

In [11]:
api_data['title'].head().str.split('-',n=1,expand=True).rename(columns={0:'artist',1:'title'})

,artist,title
0,Bing Crosby,Crosbyana
1,Bing Crosby,Crosbyana
2,Tommy Dorsey And His Clambake Seven,Tommy Dorsey And His Clambake Seven
3,"Paul Whiteman And His Orchestra, Bix Beiderbec...",Dedicated To The Memory Of Bix Beiderbecke
4,Jimmie Lunceford And His Orchestra,For Dancers Only


#### Step 3 - Unpickle Pickled Features
This step replicates the operation performed in the ``api_data`` overview, by unpickling ``genre``,``style``,``label`` and ``formats``
#### Step 4 - Clean Features with Multiple Categorical Values per Sample and High Categorical Variance
For the features ``label`` and ``artist``, it quickly becomes clear that a more sophisticated approach is necessary than just simply One-Hot Encoding.

In [12]:
label_artist_columns = api_pipe.steps[2][-1].fit_transform(api_pipe.steps[1][-1].fit_transform(api_data)).loc[:,['label','artist']]
label_artist_columns

,label,artist
0,"[Decca, The Compo Company Ltd.]",Bing Crosby
1,[Decca],Bing Crosby
2,"[Swing Classic, RCA Victor]",Tommy Dorsey And His Clambake Seven
3,[Victor],"Paul Whiteman And His Orchestra, Bix Beiderbec..."
4,"[Decca, Decca Records, Inc.]",Jimmie Lunceford And His Orchestra
...,...,...
448897,"[Production Dessinée, Disques Dessinee Distrib...",Mateo Stoneman = マテオ・ストーンマン*
448898,"[Nailuj Music, Rebel Road Studio, San Diego, R...",Julian Juno*
448899,[Rubber Frog Records],DJ Nonstick Cookware
448900,[Not On Label],Papaw (2)


First, let us identify the number of unique values for each of these columns.

In [13]:
import numpy as np

In [14]:
label_values = label_artist_columns.loc[:,'label'].apply(lambda x: 'üü'.join(x)).str.split('üü',expand=True).stack()
artist_values = label_artist_columns.loc[:,'artist']
for values, column in zip((label_values,artist_values),('label','artist')):
    print('The api_data dataset has %s unique %s values' % (len(np.unique(values)),column))

The api_data dataset has 113842 unique label values
The api_data dataset has 108872 unique artist values


As we can see from the output above, the order of magnitude of unique values for these columns prevents us from taking the standard approach in machine learning of One-Hot Encoding these columns. Let us now see if there is any possibility of reducing the number of unique values and if there the elements can be unified

In [15]:
from collections import Counter
Counter(label_values).most_common(n=20)

[('Blue Note', 17001),
 ('Columbia', 14162),
 ('Verve Records', 11287),
 ('Capitol Records', 9553),
 ('Van Gelder Studio, Englewood Cliffs, New Jersey', 9435),
 ('Atlantic', 8867),
 ('Not On Label', 8331),
 ('Capitol Records, Inc.', 7774),
 ('CBS', 7333),
 ('ECM Records', 7126),
 ('Prestige', 6943),
 ('CBS Inc.', 6862),
 ('RCA Victor', 6329),
 ('Warner Bros. Records', 6156),
 ('A&M Records', 5958),
 ('Polydor', 5681),
 ('Atlantic Recording Corporation', 5614),
 ('Impulse!', 5118),
 ('Universal Music LLC', 5089),
 ('EMI', 4933)]

From the above we can clearly see that labels have been entered under slightly different names in some cases, in particular for the  following pairs:
- Capitol Records & Capitol Records, Inc.
- CBS & CBS Inc.
Unifying these would by themselves already greatly increase the number of connected entries by homogenizing the names of the labels

In [16]:
Counter(artist_values).most_common(n=20)

[('Various', 7248),
 ('Miles Davis', 2986),
 ('John Coltrane', 2071),
 ('Frank Sinatra', 2056),
 ('Sonny Rollins', 1242),
 ('Herbie Hancock', 1124),
 ('Ella Fitzgerald', 987),
 ('The Dave Brubeck Quartet', 948),
 ('George Benson', 868),
 ('Duke Ellington', 823),
 ('Nina Simone', 822),
 ('Charles Mingus', 806),
 ('Freddie Hubbard', 758),
 ('Jimmy Smith', 745),
 ('Herbie Mann', 732),
 ('Sarah Vaughan', 698),
 ('Chet Baker', 682),
 ('Keith Jarrett', 681),
 ('Frank Zappa', 647),
 ('Thelonious Monk', 646)]

In the case of artists, it is not as clear that such cleaning would be required in order to get widespread connection between records, however, by taking a look at all entries containing 'Miles Davis', for example, we quickly observe in how many varied ways Miles Davis appears in the ``artist`` column. 

In [17]:
miles_davis_values = tuple(filter(lambda x: 'miles davis' in x.lower(), Counter(artist_values)))
miles_davis_values

('Fats Navarro / Dizzy Gillespie / Miles Davis / Kenny Dorham',
 'Miles Davis',
 'Miles Davis With Sonny Rollins',
 'Miles Davis Quartet*',
 'Miles Davis And Milt Jackson',
 'The New Miles Davis Quintet*',
 'The Miles Davis Quintet',
 'Miles Davis + 19, Gil Evans',
 'Miles Davis + 19',
 'Miles Davis, Gil Evans',
 'Miles Davis Et Son Quintette*',
 'Miles Davis All Stars',
 'Michel Legrand Featuring Miles Davis',
 'Miles Davis, Sonny Rollins',
 'Miles Davis And The Modern Jazz Giants',
 'Cannonball Adderley, Miles Davis, Hank Jones, Sam Jones, Art Blakey',
 'Miles Davis Quintet*',
 'Miles Davis Sextet*',
 'Miles Davis , Arranged And Condusted By Gil Evans',
 'The Miles Davis Sextet & The Thelonious Monk Quartet',
 'Miles Davis & The Modern Jazz Giants',
 'Miles Davis + 19 Orchestra Under The Direction Of Gil Evans',
 'Sarah Vaughan With The Miles Davis All-Stars*',
 'The Miles Davis Quintet Featuring John Coltrane',
 'The Miles Davis Sextet And Quintet* Featuring: John Coltrane, Bill Eva

Clearly, in order to reduce the dimensionality and increase the linkages between different jazz albums, it will also be crucial to clean the ``artist`` column. 

To achieve this, we apply the ``ArtistCleanReduce`` and ``LabelCleanReduce`` transformers, which clean and homogenize the text inputs, and match the resulting unique strings with one another using TF-IDF Vectorization with ngrams. In the case of ``LabelCleanReduce``, we pick the first unique element attributed to an entry, as it is deemed to be the most relevant label for the release. For both the ``artist`` and ``label`` columns, this results in a great reduction in the cardinality of entries associated with both columns, as we can see below.

In [ ]:
cleaned_text = clean_text_pipe.fit_transform(label_artist_columns)

In [ ]:
for column in ('label','artist'):
    print('The cleaned api_data has %s unique %s values' % (len(np.unique(cleaned_text.loc[:,column])),column))

In [ ]:
#Namespace Cleanup
del label_artist_columns,label_values,artist_values,miles_davis_values,cleaned_text

#### Step 5 - Remove Entries with Duplicate ``release_ids``
The results obtained from querying the Discogs.com API yielded a substantial number of duplicates for the ``release_id`` feature, as we can see from the cell below. As this is a unique identifier for each album, we will remove those entries for which a duplicate ``release_id`` is identified.

In [ ]:
print('The number of entries with duplicated release_id values is:',len(api_data[api_data['release_id'].duplicated()]))

#### Step 6 - Encode Categorical Features

For the ``column_encoding_pipe``, we turn our attention to the encoding of the ``country``, ``genre`` and ``style`` columns shown below

In [ ]:
categorical_features = Unpickler(['genre','style']).fit_transform(api_data.loc[:,['country','genre','style']])
categorical_features.head()

As we can see from the output above, we have two different encodings, as in the ``clean_text_pipe``. The ``country`` column is of a string type, whereas the ``genre`` and ``style`` features contain list-type values for each sample. It is important to know that the ``country`` attribute is provided by users themselves, whereas the ``genre`` and ``style`` values are set by Discogs and are thus more standardized

As such, for the ``genre`` and ``style`` columns, we simply need to utilize dummy encoding in order to encode the information from these features. We do this by implementing and extending the ``MultiValueCategoricalEncoder`` class which encodes the features accordingly:

In [ ]:
from lib.transformers import MultiValueCategoricalEncoder

In [ ]:
encoded_genre_style = MultiValueCategoricalEncoder('style')\
    .fit_transform(
        GenreEncoder()\
            .fit_transform(categorical_features)
)
encoded_genre_style.head()

To deal with the ``country`` feature, we take a different approach. As each country associated with a given sample contains information related to the broader region of the globe the album comes from, we implement a custom transformer called ``CountryEncoder``, which reads in the string input of the ``country`` feature, and dummy encodes the information according to which countries the album comes from, and additionally add columns outlining the regions and superregions the album release belongs to. In this case, regions refer to the the UN Geoschemes according to their M49 classification code. Superregions are used to refer to the following regions which supersede and contain within them the UN Geoscheme regions:
- Africa
- Americas
- Asia
- Europe
- Oceania
- Unknown (in case of missing information)

In [ ]:
CountryEncoder().fit_transform(categorical_features)

In [ ]:
del categorical_features, encoded_genre_style

#### Step 7 - Create and Encode Format Columns

As a next step, we must decompose the formats column such it is interpretable for our machine learning models.

In [ ]:
Unpickler('formats').fit_transform(api_data)['formats']

The ``formats`` column is saved in a dictionary type, which means that we must extract the value of each key of the dictionaries and create a new independent feature. We do this via the ``FormatEncoder`` transformer, which executes this for the four keys that are found throughout the feature, and creates the columns ``format_description``, ``format_text``,``format_name``, ``format_quantity``.

In [ ]:
formats = FormatEncoder().fit_transform(Unpickler('formats').fit_transform(api_data))[['format_name','format_quantity','format_text','format_description']]
formats

For the above features, we have two fairly cleanly formatted ones in ``format_name`` and ``format_quantity``

In [ ]:
formats['format_name'].unique()

In [ ]:
np.sort(formats['format_quantity'].unique())

While we can see from the range of values in ``format_name`` that we require no additional encoding other than One-Hot encoding prior to estimation at this stage, we see that there is a fairly long tail in the quantity of units that are associated with each album. In order to avoid these outliers from negatively impacting our predictions, we use the ``OutlierRemover`` transformer which removes values that are more than three standard deviations removed from the mean

In [ ]:
np.sort(OutlierRemover('format_quantity').fit_transform(formats)['format_quantity'].unique())

For the ``format_text``and ``format_description`` features, we utilize similar approaches as in previous sections, due to similar issues occuring in the data. For ``format_text``, we extend the ``FeatureCleanReduce`` transformer to create the ``FormatTextCleanReduce`` transformer in order to reduce the feature's cardinality

In [ ]:
len(formats['format_text'].unique())

In [ ]:
len(FormatTextCleanReduce().fit_transform(formats)['format_text'].unique())

As the cardinality of the feature is still fairly high, we will forego dummy encoding at this point and use a label encoding approach similar to the artist and label columns

For the ``format_description`` column, we observe that there is a more standardized set of values for the feature, and as such we can simply apply the ``MultiValueCategoricalEncoder`` as in previous sections for features with list datatypes

In [ ]:
MultiValueCategoricalEncoder('format_description').fit_transform(formats)

In [ ]:
del formats

#### Step 8 - Create and Encode Time Period Columns

As an additional step of feature engineering, we will utilize the ``year`` feature to encode some information about the distinct eras taking place in the history of Jazz music. This information is obtained from online sources, and encodes the following time periods and eras:
- Eras
    - Swing (1925 - 1945)
    - Modern (1940 - 1970)
    - Contemporary (1970 - 2020)
- Periods
    - Big Band (1930 - 1950)
    - Bebop (1940 - 1955)
    - Cool (1950 - 1970)
    - Fusion (1970 - 2020)
        - Dropped because it coincides exactly with the contemporary era
This simple dummy encoding of the time period incidence is achieved using the ``TimePeriodEncoder`` transformer

In [ ]:
TimePeriodEncoder().fit_transform(api_data)[[
    'year','era_modern','era_contemporary','era_swing',
    'period_big_band','period_bebop','period_cool'
]]

## Synthesis
All in all, we combine these steps through the ``api_pipe``, which yields the following output

In [ ]:
api_pipe.fit_transform(api_data)

## Extracted Data
### Loading and Overview

In [ ]:
extracted_data = load_from_pkl('extracted',path=DATA_PATH)

In [ ]:
Unpickler('track_titles').fit_transform(extracted_data)

The ``extracted_data`` DataFrame represents the data that was obtained from scraping Discogs.com release pages for the albums which were obtained through the initial querying of their API. This data is crucial in being able to build the model estimating the market value of jazz albums, as no price information is divulged by the API, and can only be programmatically obtained through web scraping. Additionally, the release pages for albums on Discogs are additionally informative in that they provide platform-specific metadata which can ultimately be used in predicting market value in the previously outlined full-information scenario. 

What follows is a description of each variable and its relevance to the project:
- ``release_id``
    - As for the ``api_data``, the release_id is the unique identifier for each album release and will form the primary key with which we can later join our data sources
- ``market_price``
    - This feature reflects the lowest price at which a given album release is available for on the Discogs.com platform. Releases which have no open listings on the platform have null values
- ``units_for_sale``
    - This feature outlines how many units of the album release are available on the Discogs.com platform
- ``average_rating``
    - Here the mean score out of 5 attributed to the album release by Discogs.com users is recorded
- ``rating_count``
    - Records the number of ratings that have been submitted for the album release
- ``last_sold``
    - Records the last time the album release was sold on the Discogs.com platform
- ``number_of_tracks``
    - This feature was generated through the process of scraping, and counts the total number of tracks that are listed for an album release
- ``running_time``
    - Like the previous feature, this one was also generated during scraping, and records the total ``running_time`` of the album in minutes
- ``lowest``, ``median``, ``highest``
    - These three features record the lowest, median and highest prices each album was sold for on the platform. 
- ``track_titles``
    - Here all the track titles of the record are recorded

## Transformation Pipeline
As for ``api_data``, we will replicate the transformation pipeline used to process the data scraped from Discogs.com release pages and go motivate each step of the pipeline.

In [ ]:
from lib.transformers import (
    ColumnRemover,
    ColumnCombiner,
    DuplicateRemover,
    NullRemover,
    StandardCountEncoder,
    LastSoldEncoder
)

In [ ]:
market_value_pipe =  Pipeline([
    #Step 3a - Creating market_value by imputing median nulls with market_price values
    ('make_market_value', ColumnCombiner('median','market_price','market_value')),
    #Step 3b - Removing samples with null values for market_value
    ('remove_nulls', NullRemover('market_value')),
    #Step 3c - Removing samples with outlier values for market_value according to 3 standard deviation rule
    ('remove_outliers', OutlierRemover('market_value'))
])

extracted_pipe = Pipeline([
    #Step 1 - Remove unnecessary columns
    ('remove_id', ColumnRemover(['id','last_sold'])),
    #Step 2 - Unpickle the track_titles features
    ('unpickle', Unpickler('track_titles')),
    #Step 3 - Creating and process the ``market_value`` feature
    ('make_market_value',market_value_pipe),
    #Step 4 - Remove any duplicates present in the release_id feature
    ('remove_duplicates', DuplicateRemover('release_id')),
    #Step 5 - Count the number of Jazz standards each album has
    ('count_standards',StandardCountEncoder('track_titles',DATA_PATH))
])

In comparison to the ``api_data`` DataFrame, the ``extracted_data`` DataFrame is far less unwieldy and messy, and instead requires only small adjustments to be made to the core features, as well as some additional transformations for feature engineering purposes.

#### Step 1 - Removing unnecessary features
We decide to remove the ``last_sold`` feature due to it having an untenable number of null values, and due to the high variation in the ranges in which albums have last been sold for the albums in question, the missing values cannot be imputed in good conscience. As such, the feature is dropped from the dataset.

In [ ]:
extracted_data['last_sold'].describe()

#### Step 2 - Unpickling ``track_titles``
Self-explanatory

#### Step 3 - Creating & processing the ``market_value`` feature
As outlined previously, we have two features in the ``extracted_data`` dataset that could conceivably be used as the dependent variable in our machine learning model to predict the market value of a given Jazz album, namely ``market_price`` and ``median``. Without missing data, it would be preferable to use the ``median`` price for the ``market_value`` of a given record, as the value of ``market_price`` is contingent on the period in which the data was scraped and may not be an accurate representation of what price the album is generally valued at in the market. Furthermore, as the ``market_price`` is the lowest price that a given record is being sold for, it could be that the reason for the price being so low is due to the poor quality of the record, a feature which is not captured in our data but is available in separate listings view of the record, which has been left out of the scope of this project in the interest of time, as it would have required the scraping another 350k web requests to the Discogs.com servers. 
However, as we can see below, we do not have access to the ``median`` price for around 150,000 albums, and as such a sole reliance on this attribute would mean we would no longer be able to incorporate these albums in our analysis. In order to minimize the amount of data that must be discarded, we take the approach of combining the ``market_price`` and ``median`` features by filling the missing values of ``median`` with the values of ``market_price``. The rationale behind this combination of features is that a missing value for ``median_price`` indicates that the album in question has never been sold on the Discogs.com platform. Why this is the case could be down to a number of reasons, such as the album being fairly niche, new on the market or not particularly popular among the types of users that Discogs.com attracts. As such, we can noisily impute the value of the record by the supply-side valuation of the album in order to reduce the number of samples that must be removed from the dataset

In [ ]:
extracted_data.loc[:,['market_price','median']].isna().sum()

In [ ]:
market_value_features = ColumnCombiner('median','market_price','market_value').fit_transform(extracted_data).loc[:,['market_value','median','market_price']]

In [ ]:
market_value_features.isna().sum()

As we can see from the above, post-imputation it is possible to keep around 100,000 samples in the dataset for training our machine learning models. Nevertheless, we must still remove the entries for which ``market_value`` is null, as well as those samples that have outlier values, which we achieve using the ``NullRemover`` and ``OutlierRemover`` transformers

In [ ]:
OutlierRemover('market_value').fit_transform(
    NullRemover('market_value').fit_transform(
        market_value_features
    )
).describe()

As we can see from the output above, by removing the nulls and outliers from the ``market_value`` feature, we manage to obtain a very similar distribution to the ``median`` feature we would have otherwise preferred from the original dataset.

In [ ]:
del market_value_features

#### Step 4 - Removing Duplicates

As in the processing of ``api_data``, it is critical that we discard the duplicate values of ``release_id`` in our dataset.

In [ ]:
print('Number of duplicates is: ',len(extracted_data)-len(DuplicateRemover('release_id').fit_transform(extracted_data)))

#### Step 5 - Count number of standards per Jazz album
In an attempt to inject our data with some additional domain knowledge, we will analyze the ``track_titles`` feature of the ``extracted_data`` DataFrame and seek to identify the number of Jazz standards that each album has on its tracklist. Jazz Standards are pieces of music which are commonly covered by artists within the Jazz community, and provide a possible entrypoint of prospective customers to purchasing the record, as they may be more likely to recognize a certain track on a record, and therefore be more incentivized to buy it. Generally, audiences enjoy hearing another artist's take on an old classic, which makes this feature of Jazz music culture a possibly informative feature to be added for our modelling. 
We accomplish this via a similar method as was employed in the ``FeatureCleanReduce`` inheriting estimators, by using TF-IDF with n-grams in order to compare the ``track_titles`` for each album with an independently sourced list of accepted jazz standards, and attempting to find high prbability matches.

In [ ]:
standards = StandardCountEncoder(path=DATA_PATH).fit_transform(
    Unpickler('track_titles').fit_transform(extracted_data))

In [ ]:
del standards

## High Level Features

In [ ]:
with np.load(os.path.join(DATA_PATH,'high_level_features.npz')) as file:
    high_level_features = pd.DataFrame(file['data'].reshape(1,-1))
    
    high_level_features['release_id'] = file['label'].reshape(1,-1)
    

In [ ]:
high_level_features

# Transforming

In [ ]:
extracted_data = extracted_pipe.fit_transform(extracted_data)
extracted_data.describe()

In [ ]:
api_data = api_pipe.fit_transform(api_data)
api_data.describe()